# Concatenating data

In [1]:
import re
import json
import pprint
import datetime
import pandas as pd

pp = pprint.PrettyPrinter(indent=4)

In [2]:
ds_emails = []
with open('../articles_email.json') as input_file:
    for line in input_file:
        ds_emails.append(json.loads(line))
        
df_emails = pd.DataFrame(ds_emails)

ds_newspapers = []

with open('../articles_expansion_hemeroteca.json') as input_file:
    for line in input_file:
        ds_newspapers.append(json.loads(line))
        
with open('../articles_cincodias.json') as input_file:
    for line in input_file:
        ds_newspapers.append(json.loads(line))
        
with open('../articles_elconfidencial.json') as input_file:
    for line in input_file:
        ds_newspapers.append(json.loads(line))
        
df_newspapers = pd.DataFrame(ds_newspapers)

print "df_emails (1): {:,}".format(df_emails.shape[0])
print "df_newspapers (0): {:,}".format(df_newspapers.shape[0])

df_emails (1): 5,146
df_newspapers (0): 15,780


In [3]:
df_emails['flag'] = 1
df_newspapers['flag'] = 0

In [4]:
df_emails.drop_duplicates(subset='url', keep='first', inplace=True)
df_newspapers.drop_duplicates(subset='url', keep='first', inplace=True)

print "df_emails (1): {:,}".format(df_emails.shape[0])
print "df_newspapers (0): {:,}".format(df_newspapers.shape[0])

df_emails (1): 4,955
df_newspapers (0): 15,768


In [5]:
frames = [df_emails, df_newspapers]
df = pd.concat(frames, axis=0, join='outer', join_axes=None, ignore_index=True)

In [6]:
print("Current number of articles: {:,}".format(df.shape[0]))
print("Number of flags = 1 ({:,})".format(df[df['flag']==1]['flag'].count()))
print("Number of flags = 0 ({:,})".format(df[df['flag']==0]['flag'].count()))

Current number of articles: 20,723
Number of flags = 1 (4,955)
Number of flags = 0 (15,768)


# Cleaning

### Articles duplicated by url

In [7]:
# removing duplicates
df.drop_duplicates(subset='url', keep='first', inplace=True)

In [8]:
print("Current number of articles: {:,}".format(df.shape[0]))
print("Number of flags = 1 ({:,})".format(df[df['flag']==1]['flag'].count()))
print("Number of flags = 0 ({:,})".format(df[df['flag']==0]['flag'].count()))

Current number of articles: 20,337
Number of flags = 1 (4,955)
Number of flags = 0 (15,382)


### Article with short text

In [9]:
# remove articles with text shorter than 200 characters
df = df.drop(df[df['text'].str.len()<200].index)

In [10]:
print("Current number of articles: {:,}".format(df.shape[0]))
print("Number of flags = 1 ({:,})".format(df[df['flag']==1]['flag'].count()))
print("Number of flags = 0 ({:,})".format(df[df['flag']==0]['flag'].count()))

Current number of articles: 19,919
Number of flags = 1 (4,880)
Number of flags = 0 (15,039)


In [11]:
# reset indexes
df.reset_index(drop=True,inplace=True)

### Text duplicates remove / relabelled to 0 

In [12]:
# text duplicates
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(10)

,text,count
0,You have requested on-line access to a Sanford...,42
1,"BIOGRAFÍA\n\nTrader, empezó su carrera profesi...",14
2,La industria de los fondos de inversión en Esp...,3
3,La madera ha jugado un papel determinante a la...,2
4,Casi dos meses después de que entrara en vigor...,2
5,Global Actuarial\n\nLas entidades financieras ...,2
6,Sede del Monte dei Paschi en Siena Giampiero S...,2
7,Más de 70 años después de la liberación del ca...,2
8,"Protesta de afectados por las claúsulas suelo,...",2
9,El presidente de la Confederación Española de ...,2


#### Text 0

In [13]:
# text 0: You have requested on-line access to a Sanford...
print("Text 0:\n")
print(df_dups.loc[0]['text'],"...")

Text 0:

(u'You have requested on-line access to a Sanford C. Bernstein report. On-line access to these reports is available only to clients of Sanford C. Bernstein & Co., LLC, Sanford C. Bernstein Limited, or Sanford C. Bernstein (Hong Kong) Limited (together, "Bernstein"). For your convenience, an additional logon id and password is not required. Simply confirm the e-mail address by which you are known to us for verification. By submitting information below, you accept the Terms of Use.\n\nPlease enter your email to verify access:', '...')


In [14]:
# get indexes corresponding to these articles
idx = df.index[df['text']==df_dups.loc[0]['text']]

print "Number of dropped articles:", len(idx)
print "Number of articles with flag = 1:", df.loc[idx]['flag'].sum()

# reset these articles to flag = 0
# df.loc[idx,'flag']=0

# drop these articles
df = df.drop(idx)

Number of dropped articles: 42
Number of articles with flag = 1: 42


In [15]:
print("Current number of articles: {:,}".format(df.shape[0]))
print("Number of flags = 1 ({:,})".format(df[df['flag']==1]['flag'].count()))
print("Number of flags = 0 ({:,})".format(df[df['flag']==0]['flag'].count()))

Current number of articles: 19,877
Number of flags = 1 (4,838)
Number of flags = 0 (15,039)


#### Text 1

In [16]:
# text 1: BIOGRAFÍA\n\nTrader, empezó su carrera profesi...
print("Text 1:\n")
print(df_dups.loc[1]['text'],"...")

Text 1:

(u'BIOGRAF\xcdA\n\nTrader, empez\xf3 su carrera profesional en GVC en 1995. Es fundador de \xc1gora Asesores Financieros EAFI y uno de los analistas t\xe9cnicos m\xe1s reputados del panorama espa\xf1ol. Ha colaborado con diversos medios de comunicaci\xf3n, entre los que se encuentra El Economista, desde su fundaci\xf3n hasta julio de 2014. Desde septiembre de 2014 se encarga de dirigir a la comunidad inversora de El Confidencial.', '...')


In [17]:
# get indexes corresponding to these articles
idx = df.index[df['text']==df_dups.loc[1]['text']]

print "Number of dropped articles:", len(idx)
print "Number of articles with flag = 1:", df.loc[idx]['flag'].sum()

# reset these articles to flag = 0
# df.loc[idx,'flag']=0

# drop these articles
df = df.drop(idx)

Number of dropped articles: 14
Number of articles with flag = 1: 0


In [18]:
print("Current number of articles: {:,}".format(df.shape[0]))
print("Number of flags = 1 ({:,})".format(df[df['flag']==1]['flag'].count()))
print("Number of flags = 0 ({:,})".format(df[df['flag']==0]['flag'].count()))

Current number of articles: 19,863
Number of flags = 1 (4,838)
Number of flags = 0 (15,025)


In [19]:
# text duplicates
df_dups = df.groupby('text').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"})
df_dups.head(10)

,text,count
0,La industria de los fondos de inversión en Esp...,3
1,La relación de la banca con sus clientes es ca...,2
2,Un agente de cambio consultando una pantalla e...,2
3,"En plena era tecnológica, donde podemos relaci...",2
4,Hace unos días asistimos a dos momentos muy in...,2
5,Ampliar foto Belén Trincado\n\nPese a que la e...,2
6,Popular reconoció ayer a los sindicatos que es...,2
7,El Senado ha suspendido de forma definitiva a ...,2
8,Popular alcanzó un máximo histórico por encima...,2
9,Sucursal de Banco Popular. Reuters\n\nFitch ha...,2


# Creating new features

* website (newspaper)
* section
* date (without time)
* day of week

In [20]:
# newspaper
df['website'] = df['url'].apply(lambda x: x.split('/')[2].replace('www.',''))

In [21]:
df[df['flag']==1].groupby('website').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"}).head(20)

,website,count
0,expansion.com,2593
1,elconfidencial.com,854
2,cincodias.elpais.com,704
3,cincodias.com,583
4,blogs.elconfidencial.com,35
5,retina.elpais.com,14
6,economia.elpais.com,6
7,euribor.com.es,3
8,bloomberg.com,3
9,vozpopuli.com,2


In [22]:
df[df['flag']==0].groupby('website').size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"}).head(20)

,website,count
0,expansion.com,9404
1,cincodias.elpais.com,3884
2,elconfidencial.com,1101
3,blogs.elconfidencial.com,295
4,vanitatis.elconfidencial.com,252
5,gentleman.elconfidencial.com,55
6,brands.elconfidencial.com,21
7,blogs.vanitatis.elconfidencial.com,8
8,alimente.elconfidencial.com,4
9,shiseido.es,1


In [23]:
# drop unnecessary data
df = df.drop(df[df['website']=="shiseido.es"].index)

In [24]:
print("Current number of articles: {:,}".format(df.shape[0]))
print("Number of flags = 1 ({:,})".format(df[df['flag']==1]['flag'].count()))
print("Number of flags = 0 ({:,})".format(df[df['flag']==0]['flag'].count()))

Current number of articles: 19,862
Number of flags = 1 (4,838)
Number of flags = 0 (15,024)


In [25]:
# section
for idx, row in df.iterrows():
    try:
        if "cincodias" in row['website']:
            df.loc[idx,'section']=row['url'].split('/')[7]
        else:
            df.loc[idx,'section']=row['url'].split('/')[3]
    except:
        pass

In [26]:
df.groupby(['website','section']).size().sort_values(0,ascending=False).reset_index().rename(columns={0:"count"}).head(25)

,website,section,count
0,expansion.com,empresas,4147
1,expansion.com,economia,2039
2,cincodias.elpais.com,companias,1077
3,expansion.com,mercados,1051
4,cincodias.elpais.com,mercados,956
5,expansion.com,aragon,785
6,elconfidencial.com,empresas,636
7,cincodias.elpais.com,empresas,618
8,expansion.com,extremadura,605
9,cincodias.com,mercados,492


In [27]:
# split the publish date to date and time and remove time because it is not changing
df['date'], df['time'] = df.publish_date.str.split(' ').str
df.drop(['time'], axis=1, inplace=True)

In [28]:
# convert date columns to datetime and create day_of_week column
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.weekday_name

In [29]:
# drop unnecessary columns
df.drop(['top_image', 'publish_date'], axis=1, inplace=True)

In [30]:
df.head(5)

,authors,keywords,summary,text,title,url,flag,website,section,date,day_of_week
0,"[Asunción Infante Fuentes, Andrés Stumpf Guirao]","[tumbar, vuelve, ibex, descenso, son, en, una,...","El Ibex cede un 1,18% y pierde el soporte clav...",Amaneció con subidas en todos los grandes parq...,El sector bancario vuelve a tumbar al Ibex: es...,http://cincodias.com/cincodias/2016/03/17/merc...,1,cincodias.com,mercados,2016-03-17,Thursday
1,[],"[ladrillo, en, desde, menor, junio, su, del, h...","El año pasado, la morosidad de este segmento h...",El pasado año el descenso de la mora tanto de ...,La morosidad del ladrillo cierra 2015 en su me...,http://www.expansion.com/empresas/banca/2016/0...,1,expansion.com,empresas,2016-03-18,Friday
2,"[Eduardo Segovia, Pedro Calvo, Contacta Al Aut...","[inversores, stanley, morgan, en, encuentro, p...",El banco de inversión Morgan stanley organiza ...,El banco de inversión Morgan stanley organiza ...,Noticias del BBVA: Los inversores señalan de q...,http://www.elconfidencial.com/mercados/2016-03...,1,elconfidencial.com,mercados,2016-03-24,Thursday
3,"[Assumpta Zorraquino, Contacta Al Autor, Carlo...","[regulación, servicios, versus, para, por, en,...",La aparición de las start-ups tecnológicas ded...,La aparición de las start-ups tecnológicas ded...,Fintech: Innovación versus regulación,http://blogs.elconfidencial.com/mercados/tribu...,1,blogs.elconfidencial.com,mercados,2016-03-24,Thursday
4,"[Pedro Calvo, Kike Vázquez, Daniel Lacalle, Vi...","[europea, pierde, por, en, y, valor, su, del, ...",Como el que han pagado en el último año las gr...,"Son los bancos españoles. Y los italianos, por...",Tipos de interés: La banca europea pierde un t...,http://www.elconfidencial.com/empresas/2016-03...,1,elconfidencial.com,empresas,2016-03-29,Tuesday


# Train / Test

Preparing the train / test dataset for modelling:

* __`TRAIN`__: articles published before 2017-10-01 (+ articles without date).
* __`TEST`__: articles published from 2017-10-01 on.

In [31]:
# Train examples span from the very start up until 2018. 
df_train = df[(df['date']<'2017-10-01') | (df['date'].isnull())]
df_test  = df[df['date']>='2017-10-01']

In [32]:
print "Number of null dates in train: {:,}".format(df_train[df_train['date'].isnull()]['url'].count())
print "Number of null dates in test: {:,}".format(df_test[df_test['date'].isnull()]['url'].count())

Number of null dates in train: 31
Number of null dates in test: 0


In [33]:
print "-"*50, "\nTRAIN (< 2017-10-01 + None)\n", "-"*50
print "Flag:\t1\t\t0"
print "Nobs:\t{:,}".format(df_train[df_train['flag']==1]['url'].count()),"\t\t{:,}".format(df_train[df_train['flag']==0]['url'].count())

print "\n","-"*50, "\nTEST (>= 2017-10-01)\n", "-"*50
print "Flag:\t1\t\t0"
print "Nobs:\t{:,}".format(df_test[df_test['flag']==1]['url'].count()),"\t\t{:,}".format(df_test[df_test['flag']==0]['url'].count())

-------------------------------------------------- 
TRAIN (< 2017-10-01 + None)
--------------------------------------------------
Flag:	1		0
Nobs:	3,678 		10,959

-------------------------------------------------- 
TEST (>= 2017-10-01)
--------------------------------------------------
Flag:	1		0
Nobs:	1,160 		4,065


In [34]:
# shuffle datasets
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_test = df_test.sample(frac=1).reset_index(drop=True)

In [35]:
# create id
df_train['id'] = df_train.index + 1
df_test['id'] = df_test.index + 1

In [36]:
df_test_nosolution = df_test.drop('flag', axis=1)
df_test_solution = df_test[['id', 'flag']]

# Dump to JSON

In [37]:
df_train.to_json('./train.json', force_ascii=False)
df_test_nosolution.to_json('./test.json', force_ascii=False)
df_test_solution.to_json('./test_solution.json', force_ascii=False)

# Dump to CSV

In [38]:
df_train.to_csv('./train.csv', sep=",", na_rep="", mode="w", index=False, encoding='utf-8')
df_test_nosolution.to_csv('./test.csv', sep=",", na_rep="", mode="w", index=False, encoding='utf-8')
df_test_solution.to_csv('./test_solution.csv', sep=",", na_rep="", mode="w", index=False, encoding='utf-8')

In [39]:
import random

df_sample = df_test_solution[['id']]

for idx, row in df_sample.iterrows():
    df_sample.loc[idx, 'pred'] = random.random()

In [40]:
df_sample.to_csv('./random_sample.csv', sep=",", na_rep="", mode="w", index=False, encoding='utf-8')

In [41]:
df_sample_solution = df_test_solution.rename(columns={'flag':'pred'})

In [42]:
df_sample_solution.to_csv('./solution_sample.csv', sep=",", na_rep="", mode="w", index=False, encoding='utf-8')